In [1]:
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import Counter

class PRLStructure(Structure):
    """A pymatgen Structure object, with some customizations for ESPEI.
    """

    def __init__(self, *args, **kwargs):
        """Create a Structure object, with some customizations for ESPEI

        Parameters
        ----------
        args :
            args to pass to Structure
        sublattice_configuration : [[str]]
            Sublattice configuration  e.g. `[['Fe', 'Ni'], ['Fe']]`.
        sublattice_occupancies : [[float]]
            Fraction of the sublattice each element in the configuration  has e.g. `[[0.3333, 0.6666], [1]]`.
        sublattice_site_ratios : [float]
            Ratios of sublattice multiplicity  e.g. `[3, 1]`.
        kwargs :
            kwargs to pass to Structure
        """
        self.sublattice_configuration = kwargs.pop('sublattice_configuration', None)
        self.sublattice_occupancies = kwargs.pop('sublattice_occupancies', None)
        self.sublattice_site_ratios = kwargs.pop('sublattice_site_ratios', None)
        self.wyckoff_sites = kwargs.pop('wyckoff_sites', None)
        super(PRLStructure, self).__init__(*args, **kwargs)

    def __eq__(self, other):
        """
        self and other are equivalent if the sublattice models are equal

        Parameters
        ----------
        other : PRLStructure
        """
        if not isinstance(other, PRLStructure):
            return False
        subl_config = self.sublattice_configuration == other.sublattice_configuration
        subl_site_ratios = self.sublattice_site_ratios == other.sublattice_site_ratios
        subl_occupancies = self.sublattice_occupancies == other.sublattice_occupancies
        wyckoff_sites = self.wyckoff_sites == other.wyckoff_sites
        return subl_config and subl_site_ratios and subl_occupancies and wyckoff_sites

    @property
    def espei_sublattice_configuration(self):
        """
        Return ESPEI-formatted sublattice model [['a', 'b'], 'a'] for the concrete case
        """
        # short function to convert [['A', 'B'], ['A']] to [['A', 'B'], 'A'] as in ESPEI format
        canonicalize_sublattice = lambda sl: sl[0] if len(sl) == 1 else sl
        return [canonicalize_sublattice(sl) for sl in self.sublattice_configuration]

    @property
    def espei_sublattice_occupancies(self):
        """
        Return ESPEI-formatted sublattice occupancies [[0.3333, 0.6666], 1] for the concrete case
        """
        # short function to convert [[0.3333, 0.6666], [1]] to [[0.3333, 0.6666], 1] as in ESPEI format
        canonicalize_sublattice = lambda sl: sl[0] if len(sl) == 1 else sl
        return [canonicalize_sublattice(sl) for sl in self.sublattice_occupancies]

    def as_dict(self, verbosity=1, fmt=None, **kwargs):
        d = super(PRLStructure, self).as_dict(verbosity=verbosity, fmt=fmt, **kwargs)
        d['sublattice_configuration'] = self.sublattice_configuration
        d['sublattice_occupancies'] = self.sublattice_occupancies
        d['sublattice_site_ratios'] = self.sublattice_site_ratios
        return d

    @classmethod
    def from_dict(cls, d, fmt=None):
        struct = super(PRLStructure, cls).from_dict(d, fmt=fmt)
        struct.sublattice_configuration = d.get('sublattice_configuration')
        struct.sublattice_occupancies = d.get('sublattice_occupancies')
        struct.sublattice_site_ratios = d.get('sublattice_site_ratios')
        return struct


    @classmethod
    def from_structure(cls, structure, equivalent_sites=None):
        """

        Parameters
        ----------
        structure : pymatgen.Structure
        equivalent_sites : list of lists
            List of Wyckoff sites that are treated as the same sublattice, e.g. [['b', 'f']] will
            give combine Wyckoff site 'b' and Wyckoff site 'f' into one sublattice. Putting the same
            Wyckoff site in multiple equivalent groups will produce undefined results.

        Returns
        -------
        PRLStructure
        """
        struct = PRLStructure.from_dict(structure.as_dict())
        # normalize the input structure to a pure element to get Wyckoff sites
        structure = Structure.from_dict(structure.as_dict())
#        structure.replace_species({sp.name: "H" for sp in structure.species})
        sga = SpacegroupAnalyzer(structure)
        wyckoff_sites = sga.get_symmetry_dataset()['wyckoffs']
        equal_atom = sga.get_symmetry_dataset()['equivalent_atoms']
        num_wyckoff_sites = sorted(set(wyckoff_sites))
        num_eq_atom=sorted(set(equal_atom))
        if num_wyckoff_sites == num_eq_atom:
            true_sites = wyckoff_sites
        else:
            subl_wyckoff_name=[]
            for i in num_eq_atom:
                subl_wyckoff_name.append(wyckoff_sites[i])
            replace_list=[]
            original_list=[]
            for i in subl_wyckoff_name:
                original_list.append(i)
                if i in replace_list:
                    count_subl=dict(Counter(original_list))
                    j=i+str(count_subl[i])
                else:
                    j=i
                replace_list.append(j)
            replace_dict=dict(zip(num_eq_atom, replace_list))
            true_sites=[replace_dict[i] for i in equal_atom]
        true_sublattices = sorted(set(true_sites))
        if equivalent_sites is not None:
            # transform the true sublattices by combining equivalent sites
            combined_sublattices = ['-'.join(sorted(sites)) for sites in equivalent_sites]
            def match_subl(candidate):
                for subl in combined_sublattices:
                    # if the candidate site is in the combined sublattice, return the combined sublattice
                    if candidate in subl:
                        return subl
                # no match found
                return candidate

            new_subl_model = sorted(set([match_subl(subl) for subl in true_sublattices]))
        else:
            new_subl_model = true_sublattices

        #ratios = [sum([1 if site in subl else 0 for site in wyckoff_sites]) for subl in new_subl_model]
        config = []
        occ = []
        ratios = []
        for subl in new_subl_model:
            species_frequency_dict = {}
            for site, wyckoff_site in zip(struct.sites, true_sites):
                if '-' in subl:
                    if wyckoff_site in subl:
                        species=site.specie.name.upper()
                        species_frequency_dict[species] = species_frequency_dict.get(species, 0) + 1
                else:
                    if wyckoff_site == subl:
                        species = site.specie.name.upper()
                        species_frequency_dict[species] = species_frequency_dict.get(species, 0) + 1
            total_subl_occupation = sum(species_frequency_dict.values())
            subl_species = sorted(set(species_frequency_dict.keys()))
            subl_occpancy = [species_frequency_dict[sp]/total_subl_occupation for sp in subl_species]
            config.append(subl_species)
            occ.append(subl_occpancy)
            ratios.append(total_subl_occupation)
        #config = [sorted(set([site.specie.name for site, wyckoff in  if wyckoff in subl])) for subl in new_subl_model]

        struct.sublattice_configuration = config
        struct.sublattice_occupancies = occ
        struct.sublattice_site_ratios = ratios
        struct.wyckoff_sites = sorted(subl_wyckoff_name)
        return struct

    @staticmethod
    def reindex_sublattice(new_indices, subl_model, subl_occupancies, subl_site_ratios):
        """
        Re-index the passed sublattice model, occupancies and site ratios according to the new index.

        Parameters
        ----------
        new_indices : [int]
            List of indicies corresponding to sublattices. There should be no duplicates. Specifically,
            sorted(new_indices) == list(range(len(subl_model))
        subl_model : [[str]]
            Sublattice configuration  e.g. `[['Fe', 'Ni'], ['Fe']]`.
        subl_occupancies : [[float]]
            Fraction of the sublattice each element in the configuration  has e.g. `[[0.3333, 0.6666], [1]]`.
        subl_site_ratios : [float]
            Ratios of sublattice multiplicity  e.g. `[3, 1]`.

        Returns
        -------
        tuple
            Tuple of (sublattice model, occupancies, site ratios) that have been re-indexed

        Examples
        --------

        >>> PRLStructure.reindex_sublattice([1, 0], [['Al', 'Ni'], ['Al']], [[0.333, 0.666], [1]], [3, 1])
        ([['Al'], ['Al', 'Ni']], [[1], [0.333, 0.666]], [1, 3])
        """
        if sorted(new_indices) != list(range(len(subl_model))):
            raise ValueError('Passed re-indexing indicies ({}) do not match the sublattice model indices ({}).'.format(new_indices, list(range(len(subl_model)))))
        new_subl_model = [subl_model[i] for i in new_indices]
        new_subl_occupancies = [subl_occupancies[i] for i in new_indices]
        new_subl_site_ratios = [subl_site_ratios[i] for i in new_indices]
        return (new_subl_model, new_subl_occupancies, new_subl_site_ratios)


    def reindex(self, new_indices):
        """
        Re-index the instance sublattice model, occupancies and site ratios according to the new index.

        Parameters
        ----------
        new_indices : [int]
            List of indicies corresponding to sublattices. There should be no duplicates. Specifically,
            sorted(new_indices) == list(range(len(subl_model))

        """
        self.sublattice_configuration, self.sublattice_occupancies, self.sublattice_site_ratios = \
            PRLStructure.reindex_sublattice(new_indices, self.sublattice_configuration, self.sublattice_occupancies, self.sublattice_site_ratios)


In [2]:
"""
Tools for substituting structures and generating metadata
"""

from copy import deepcopy
def sort_x_by_y(x, y):
    """Sort a list of x in the order of sorting y"""
    return [xx for _, xx in sorted(zip(y, x), key=lambda pair: pair[0])]

def canonicalize_config(configuration, occupancies):
    """
    Return canonicalized (sorted) configurations and occupancies.

    Parameters
    ----------
    configuration : list of lists
        DFTTK-style configuration
    occupancies :
        DFFTK-style occupancies

    Returns
    -------
    tuple
        Tuple of canonical (configuration, occupancies)

    Notes
    -----
    The sublattice ordering is preserved, but the species within a sublattice are sorted.

    Examples
    --------
    >>> canonicalize_config([['Fe', 'Ni'], ['Fe']], [[0.25, 0.75], [1.0]])  # already canonical
    ([['Fe', 'Ni'], ['Fe']], [[0.25, 0.75], [1.0]])
    >>> canonicalize_config([['Cu'], ['Mg']], [[1.0], [1.0]])  # already canonical
    ([['Cu'], ['Mg']], [[1.0], [1.0]])
    >>> canonicalize_config([['Cu', 'Mg']], [[0.9, 0.1]])  # already canonical
    ([['Cu', 'Mg']], [[0.9, 0.1]])
    >>> canonicalize_config([['Cu', 'Mg']], [[0.1, 0.9]])  # already canonical
    ([['Cu', 'Mg']], [[0.1, 0.9]])
    >>> canonicalize_config([['Ni', 'Fe'], ['Fe']], [[0.75, 0.25], [1.0]])
    ([['Fe', 'Ni'], ['Fe']], [[0.25, 0.75], [1.0]])
    >>> canonicalize_config([['Ni', 'Fe'], ['Fe', 'Cr', 'Ni']], [[0.75, 0.25], [0.1, 0.2, 0.7]])
    ([['Fe', 'Ni'], ['Cr', 'Fe', 'Ni']], [[0.25, 0.75], [0.2, 0.1, 0.7]])

    """
    new_occupancies = [sort_x_by_y(occ, config) for occ, config in zip(occupancies, configuration)]
    new_configuration = [sorted(config) for config in configuration]
    return (new_configuration, new_occupancies)

def get_density_from_pt(ele_list):
    """
    Get density(g/cm^3) from periodictable package

    Parameters
    ----------
        ele_list : list/dict
            The list of elements, e.g. ['Nb', 'Ti']/{'Nb': 3, 'Ti': 1}
    Returns
    -------
        density_dict : dict
            Dictionary of {element: density}, e.g. {'Nb': 8.57, 'Ti': 4.507}. 
    Examples
    --------
    >>> get_density_from_pt(['Nb', 'Ti'])
    {'Nb': 8.57, 'Ti': 4.507}
    """
    from pymatgen.core.periodic_table import Element
    #import periodictable as pt
    density_dict = {}
    for ele in ele_list:
        density_dict[ele] = float(Element(ele).density_of_solid)/1000.
    return density_dict

def get_ele_list_from_struct(struct):
    """
    Get elements list from pymatgen structure objective

    Parameters
    ----------
        struct : pymatgen objective
            The structure
    Returns
    -------
        ele_list : [str]
            The list of elements
    """
    ele_list = []
    for ele in struct.species:
        ele_list.append(str(ele))
    return ele_list

def scale_struct(struct):
    """Scale the structure according to the weighted average density of each element.

    Parameters
    ----------
    struct : pymatgen.Structure
    #density_dict : dict
    #    Dictionary of {element: density}, e.g. {'Fe': 9, 'Ti': 4}. The units do
    #    not matter as long as the densities in the dict are internally consistent.
    # This parameters is canceled by using periodictable module in pymatgen.core

    Returns
    -------
    pymatgen.Structure
        Modifies the structure in place, but also returns for convenience.

    """
    species_amnt_dict = struct.composition.get_el_amt_dict()  # dict of {'V': 10.0, 'Ni': 30.0}
    density_dict = get_density_from_pt(species_amnt_dict)
    # densities is dict of densities, {'V': 6.313, 'Ni': 9.03}
    expected_density = float(sum([density_dict[species]*amnt for species, amnt in species_amnt_dict.items()]))/struct.composition.num_atoms
    current_density = struct.density
    current_volume = struct.volume
    expected_volume = current_volume/expected_density*current_density
    struct.scale_lattice(float(expected_volume))
    return struct

def gen_replacement_dict(old_config, new_config):
    """Create a pymatgen replacement dict based on old and new sublattice configurations.

    Shapes of the config lists must match.

    Parameters
    ----------
    old_config : list
        DFTTK style configuration that will be replaced
    new_config : list
        DFTTK style configuration that the new structure will have

    Returns
    -------
    dict
        Dict of {element_to_replace: new_element}
    """
    replacement_dict = {}
    for new_subl, old_subl in zip(new_config, old_config):
        for new_atom, old_atom in zip(new_subl, old_subl):
            replacement_dict[old_atom] = new_atom
    return replacement_dict


def substitute_configuration(template_structure, template_config, config, check_sorting=True):
    """
    Replace the species in the template structure by switching the template_config elements for the config elements.

    Scales the structure according to the weighted element densities. Shapes of
    the config lists must match.

    Parameters
    ----------
    template_structure : pymatgen.Structure
        Structure with species that will be replaced
    template_config : list
        DFTTK style configuration that will be replaced
    config : list
        DFTTK style configuration that the new structure will have
    #density_dict : dict
    #    Dictionary of {element: density}, e.g. {'Fe': 9, 'Ti': 4}. The units do
    #    not matter as long as the densities in the dict are internally consistent.
    check_sorting : bool
        If True, will check that all sublattices are correctly sorted in the target config

    Returns
    -------
    pymatgen.Structure
        A new Structure object (the original is not modified so it can be reused in loops).
    """
    #if check_sorting:
    #    for subl in config:
    #        if subl != list(sorted(subl)):
    #            raise ValueError("Configuration {} is not in sorted order. "
    #                             "See information on DFTTK configurations in the docs.".format(config))
    struct = deepcopy(template_structure)
    struct.replace_species(gen_replacement_dict(template_config, config))
    scale_struct(struct)
    return struct



def substitute_configuration_with_metadata(template_structure, template_config, config, occupation, phase_name, site_ratios):
    """
    Replace the species in the template structure by switching the template_config elements for the config elements.

    Scales the structure according to the weighted element densities. Shapes of
    the config lists must match.

    Wrapper around subsitute_configuration that returns a tuple of the structure and metadata.

    Parameters
    ----------
    template_structure : pymatgen.Structure
        Structure with species that will be replaced
    template_config : list
        DFTTK style configuration that will be replaced
    config : list
        DFTTK style configuration that the new structure will have
    #density_dict : dict
    #    Dictionary of {element: density}, e.g. {'Fe': 9, 'Ti': 4}. The units do
    #    not matter as long as the densities in the dict are internally consistent.
    occupation : list
        DFTTK style occupancy fractions. Must match the shape to config and template_config.
    phase_name : str
        Name of the phase
    site_ratios : list
        Sublattice site ratios, 1-d list.

    Returns
    -------
    (pymatgen.Structure, dict)
        Tuple of a new Structure object (the original is not modified so it can be reused in loops) and a dict of metadata
    """
    struct = substitute_configuration(template_structure, template_config, config, check_sorting=False)
    config, occupation = canonicalize_config(config, occupation)
    metadata = {'phase_name': phase_name, 'sublattice': {'configuration': config, 'occupancies': occupation, 'site_ratios': site_ratios}}
    return struct, metadata


In [3]:
"""Convienence functions for building endmembers"""
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.periodic_table import Element
from itertools import permutations, product, chain
from collections import Counter

def get_sublattice_information(structure, use_equivalent_atom=False):
    """
    Return defined sublattice_name based on wyckoff positions and other sublattice information

    Parameters
    ----------
    structure : pymatgen.Structure

    use_equivalent_atom: 
        From pymatgen function, it shows the wyckoff letter for each site. Some sites with the same
        wyckoff letters may still be symmetric inequivalent due to their coordinates. Here equivalent atom
        setting will distinguish symmetry inequivalent sites and provide more information for build up sublattice.
        The default setting is False. If you would like to get information based on equivalent atoms, use True.

    Returns
    -------
    true_sublattice_sites: list
        List of wyckoff letters for each site

    subl_model_name: list
        Name for each sublattice based on the name of wyckoff sites
    
    sublattice_site_ratio: list
        Site ratio for each sublattice
        
    Note:
    Need to consider order-disorder case
    """
    structure.replace_species({sp.name: "H" for sp in structure.species})
    sga = SpacegroupAnalyzer(structure)
    wyckoff_sites = sga.get_symmetry_dataset()['wyckoffs']
    equal_atom = sga.get_symmetry_dataset()['equivalent_atoms']
    num_wyckoff_sites = sorted(set(wyckoff_sites))
    num_eq_atom=sorted(set(equal_atom))
    if len(num_wyckoff_sites)!=len(num_eq_atom):
        print('Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmerty information and choose the sublattice model.')
    if use_equivalent_atom==True:
        sub_wyckoff_name=[]
        for i in num_eq_atom:
            sub_wyckoff_name.append(wyckoff_sites[i])
        replace_list=[]
        original_list=[]
        for i in sub_wyckoff_name:
            original_list.append(i)
            if i in replace_list:
                count_subl=dict(Counter(original_list))
                j=i+str(count_subl[i])
            else:
                j=i
            replace_list.append(j)
        replace_dict=dict(zip(num_eq_atom, replace_list))
        true_sublattice_sites=[replace_dict[i] for i in equal_atom]
    else:
        true_sublattice_sites = wyckoff_sites
    subl_model_name = sorted(set(true_sublattice_sites))
    sublattice_site_ratio=[]
    for i in subl_model_name:
        j=true_sublattice_sites.count(i)
        sublattice_site_ratio.append(j)
    return true_sublattice_sites, subl_model_name, sublattice_site_ratio

def get_templates(structure, wyckoff_site_list, subl_model_name, equivalent_sites=None):
    """
    Return templates of structure and configuration for substitution of endmembers

    Parameters
    ----------
    structure : pymatgen.Structure

    wyckoff_site_list: list
        List of wyckoff letter for each site, can be get from function get_sublattice_information. 
        If input manually, please make sure the order matches with the order of postions in pymatgen.Structrue

    subl_model_name: list
        Name for each sublattice based on the name of wyckoff sites.

    equivalent_sites: dict
        Set the equivalent sites when needed. For example, {'b': 'a'} means merge site 'a' and 'b' to the same sublattice 'a'.
    
    Returns
    -------
    template_structure: pymatgen.Structure

    template_configuration: list
        Template configuration from the template structure and sublattice model.
    """
    if equivalent_sites is not None:
        true_sublattice_sites=[equivalent_sites[i] if i in equivalent_sites else i for i in wyckoff_site_list]
        rep_sublattices=[equivalent_sites[i] if i in equivalent_sites else i for i in subl_model_name]
    else:
        true_sublattice_sites=wyckoff_site_list
        rep_sublattices=subl_model_name
    true_sublattices=sorted(set(rep_sublattices))
    site_list=[]
    dict_struct=structure.as_dict()
    for i in true_sublattice_sites:
        site_dict={'sublattice_sites':i}
        site_list.append(site_dict)
    for i in range(0,len(dict_struct['sites'])):
        dict_struct['sites'][i]['properties'].update(site_list[i])
        dict_struct['sites'][i]['species'][0]['element']=site_list[i]['sublattice_sites']
    template_configuration=[]
    for i in range(0,len(true_sublattices)):
        for element in dict_struct['sites']:
            if element['species'][0]['element']==true_sublattices[i]:
                element['species'][0]['element'] = str(Element.from_Z(i+1))
        template_configuration.append(str(Element.from_Z(i+1)))
    template_structure=Structure.from_dict(dict_struct)
    return template_structure, template_configuration


def get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration):
    """
    Return endmembers of the structure

    Parameters
    ----------
    template_structure: pymatgen.Structure

    template_configuration: list
        Configuration in the template structure, should be consitent with sublattice model, e.g., one unique
        element for each sublattice. If input manually, make sure be consistent with the sublattice_model_name 

    sublattice_configuration: list
        List of configurations for each sublattice

    Returns
    -------
    endmembers: list 
        List of structures in pymatgen.Structure
    """
    element_dict=dict(map(lambda x, y: [x, y], template_configuration, sublattice_configuration))
    subl_combination=[]
    for comb in product(*map(element_dict.get, sorted(element_dict))):
        subl_combination.append(list(comb))
    endmembers=[]
    for i in subl_combination:
        endmembers.append(substitute_configuration(template_structure, [template_configuration], [i]))
    return subl_combination, endmembers


In [4]:
"""Convienence functions for building dilute structures"""
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.periodic_table import Element
from itertools import permutations, product, chain
from collections import Counter
import copy

def dilute_substitution(endmembers, sublattice_dict, supercell_matrix=None):
    """
    Return dilute structures base on endmembers and elements

    Parameters
    ----------
    endmembers: list of endmember structures from endmember.py

    sublattice_dict: dict of elements for substitutions for each sublattice, e.g., {'a': ['Hf', 'Mo'], 'f': ['Hf', 'Mo'], 'h': ['Hf', 'Mo']}
    
    supercell_matrix: list of matrix to make supercell, e.g., [2,2,2], [[2,1,0],[0,3,0],[0,0,1]], default is none

    Returns
    -------
    dilute structures: list
    
    dilute sublattice configurations
    """
    dilute_strs=[]
    dilute_conf=[]
    structures=copy.deepcopy(endmembers)
    if supercell_matrix is not None:
        for i in structures:
            i.make_supercell(supercell_matrix)
    for strs in structures:
        dictstr=strs.as_dict()
        sub_lattice_list=[]
        for i in dictstr['sites']:
            j=i['properties']['sublattice_sites']
            sub_lattice_list.append(j)
        num=[]
        for i in range(0, len(sub_lattice_list)):
            if i != len(sub_lattice_list):
                if sub_lattice_list[i] != sub_lattice_list[i-1]:
                    num.append(i)          
        for i in num:
            dictstr=strs.as_dict()
            old_ele=dictstr['sites'][i]['species'][0]['element']
            site_sub=dictstr['sites'][i]['properties']['sublattice_sites']
            sublattice_num=sub_lattice_list[i]
            element_dict=sublattice_dict[sublattice_num]
            for j in element_dict:
                if j != 'fix':
                    if old_ele != j:
                        dictstr['sites'][i]['species'][0]['element']=j
                        dilute_strs.append(Structure.from_dict(dictstr))
                                      
    for dilute in dilute_strs:
        comb={}
        dictdilute=dilute.as_dict()
        for i in num:
            sfirst_ele=dictdilute['sites'][i]['species'][0]['element']
            ssecond_ele=dictdilute['sites'][i+1]['species'][0]['element']
            site_sub=dictdilute['sites'][i]['properties']['sublattice_sites']
            if sfirst_ele == ssecond_ele:
                comb[site_sub]=sfirst_ele
            else:
                conflist=list()
                conflist.append(sfirst_ele)
                conflist.append(ssecond_ele)
                comb[site_sub]=conflist
        dilute_conf.append(comb)
    return dilute_strs, dilute_conf


In [5]:
def get_structures_from_database(db, prototype, subl_model, subl_site_ratios):
    """Returns a list of Structure objects from the db that match the criteria.

    The returned list format supports matching SQS to phases that have multiple solution sublattices
    and the inclusion of higher and lower ordered SQS that match the criteria.

    Parameters
    ----------
    db : tinydb.database.Table
        TinyDB database of the SQS database
    prototype : str
        Prototype symbol as in ATAT sqsdb, e.g. 'GAMMA_L12'.
    subl_model : [[str]]
        List of strings of species names. This sublattice model can be of higher dimension than the SQS.
        Outer dimension should be the same length as subl_site_ratios.
    subl_site_ratios : [[float]]
        Scalar multiple of site ratios of each sublattice. e.g. [1, 2] will match [2, 4] and vice
        versa. Outer dimension should be the same length as subl_model.

    Returns
    -------
    [AbstractSQS]
        Abstract SQSs that match the symmetry and sublattice model.
    """
    def lists_are_multiple(l1, l2):
        """
        Returns True if list a is a multiple of b or vice versa

        Parameters
        ----------
        l1 : [int]
        l2 : [int]

        Returns
        -------
        bool

        """
        # can we compare these two lists?
        if len(l1) != len(l2):
            return False
        for a, b in [(l1, l2), (l2, l1)]:  # check the reverse of the lists too
            # see if all a are perfectly divisible by all b
            if all([(x % y == 0) for x, y in zip(a, b)]):
                # see if all have the same multiple
                if len(set([x/y for x, y in zip(a, b)])) == 1:
                    return True
        return False

    from tinydb import where
    results = db.search((where('prototype') == prototype) &
                        (where('sublattice_site_ratios').test(
                         lambda x: (lists_are_multiple([sum(subl) for subl in x], subl_site_ratios))))
              )
    return results

In [6]:
"""
The sqs module handles converting abstract SQS Structure objects to concrete structures.

The SQS are regular pymatgen Structures with the species named according to sublattice and species type.
These species in pymatgen Structures are named to `Xab`, which corresponds to atom `B` in sublattice `a`.
"""

from __future__ import division

import copy
import itertools

import pymatgen as pmg
from pymatgen.core import Structure


class AbstractSQS(Structure):
    """A pymatgen Structure with special features for SQS.
    """

    def __init__(self, *args, **kwargs):
        """Create a SQS object

        Parameters
        ----------
        args :
            args to pass to Structure
        sublattice_model : [[str]]
            Abstract sublattice model in the ESPEI style, e.g. `[['a', 'b'], ['a']]`.
        sublattice_names : [[str]]
            Names of the sublattices, or the second character in the species names, e.g. `['a', 'c']`.
        kwargs :
            kwargs to pass to Structure
        """
        self.sublattice_model = kwargs.pop('sublattice_model', None)
        self._sublattice_names = kwargs.pop('sublattice_names', None)
        super(AbstractSQS, self).__init__(*args, **kwargs)

    @property
    def normalized_sublattice_site_ratios(self):
        """Return normalized sublattice site ratio. E.g. [[0.25, 0.25], [0.1666, 0.1666, 0.1666]]
        """
        subl_model = self.sublattice_model
        subl_names = self._sublattice_names
        comp_dict = self.composition.as_dict()
        site_ratios = [[comp_dict['X'+name+e+'0+']/self.num_sites for e in subl] for subl, name in zip(subl_model, subl_names)]
        return site_ratios

    @property
    def sublattice_site_ratios(self):
        """Return normalized sublattice site ratio. E.g. [[0.25, 0.25], [0.1666, 0.1666, 0.1666]]
        """
        subl_model = self.sublattice_model
        subl_names = self._sublattice_names
        comp_dict = {k: int(v) for k, v in self.composition.reduced_composition.as_dict().items()}
        site_ratios = [[comp_dict['X'+name+e+'0+'] for e in subl] for subl, name in zip(subl_model, subl_names)]
        return site_ratios

    def get_concrete_sqs(self, subl_model, scale_volume=True):
        """Modify self to be a concrete SQS based on the sublattice model.

        Parameters
        ----------
        subl_model : [[str]]
            List of strings of species names. Must exactly match the shape of self.sublattice_model.
            **Note that order does matter!** [["Al", "Fe"]] and [["Fe", "Al"]] will produce different results!
        scale_volume : bool
            If True, scales the volume of the cell so the ions have at least their minimum atomic radii between them.
        """
        def _subl_error():
            raise ValueError('Concrete sublattice model {} does not match size of abstract sublattice model {}'.format(subl_model, self.sublattice_model))
        if len(subl_model) != len(self.sublattice_model):
            _subl_error()

        # build the replacement dictionary and the site ratios
        # we have to look up the sublattice names to build the replacement species names
        replacement_dict = {}
        site_occupancies = [] # list of [{'FE': 0.3333, 'NI': 0.6666}, {'FE': 1}] for [['FE', 'NI'], ['FE]]
        for abstract_subl, concrete_subl, subl_name, subl_ratios in zip(self.sublattice_model, subl_model, self._sublattice_names, self.sublattice_site_ratios):
            if len(abstract_subl) != len(concrete_subl):
                _subl_error()
            sublattice_ratio_sum = sum(subl_ratios)
            sublattice_occupancy_dict = {}
            for abstract_specie, concrete_specie, site_ratio in zip(abstract_subl, concrete_subl, subl_ratios):
                specie = 'X' + subl_name + abstract_specie
                replacement_dict[specie] = concrete_specie
                sublattice_occupancy_dict[concrete_specie] = sublattice_occupancy_dict.get(concrete_specie, 0) + site_ratio/sublattice_ratio_sum
            site_occupancies.append(sublattice_occupancy_dict)

        # create a copy of myself to make the transformations and make them
        self_copy = copy.deepcopy(self)
        self_copy.replace_species(replacement_dict)

        if scale_volume:
            fractional_comp = dict(self_copy.composition.fractional_composition)
            estimated_density = 0
            for component in self_copy.composition.elements :
                temp = pmg.core.periodic_table.Element(component).data['Density of solid']
                density = float(temp.split(' ')[0])
                estimated_density += (fractional_comp[component] * density)/1000
            self_copy.scale_lattice(float((self_copy.volume/estimated_density)*self_copy.density))

        # finally we will construct the SQS object and set the values for the canonicalized
        # sublattice configuration, site ratios, and site occupancies

        # first, canonicalize the sublattice model, e.g. [['FE', 'FE'], ['NI']] => [['FE'], ['NI']]
        sublattice_configuration = [sorted(set(subl)) for subl in subl_model]
        # construct the sublattice occupancies for the model
        sublattice_occupancies = [[occupancies[specie] for specie in subl] for occupancies, subl in zip(site_occupancies, sublattice_configuration)]
        # sum up the individual sublattice site ratios to the total sublattice ratios.
        # e.g [[0.25, 0.25], [0.1666, 0.1666, 0.1666]] => [0.5, 0.5]
        site_ratios = [sum(ratios) for ratios in self.sublattice_site_ratios]

        # create the SQS and add all of these properties to our SQS
        concrete_sqs = PRLStructure.from_sites(self_copy.sites)
        concrete_sqs.sublattice_configuration = sublattice_configuration
        concrete_sqs.sublattice_occupancies = sublattice_occupancies
        concrete_sqs.sublattice_site_ratios = site_ratios
        return concrete_sqs


    def get_endmember_space_group_info(self, symprec=1e-2, angle_tolerance=5.0):
        """
        Return endmember space group info..

        Args:
            symprec (float): Same definition as in SpacegroupAnalyzer.
                Defaults to 1e-2.
            angle_tolerance (float): Same definition as in SpacegroupAnalyzer.
                Defaults to 5 degrees.

        Returns:
            spacegroup_symbol, international_number
        """
        endmember_subl = [['X' + subl_name for _ in subl] for subl, subl_name in
                          zip(self.sublattice_model, self._sublattice_names)]
        # we need to replace the abstract names with real names of species.
        endmember_speices = {specie for subl in endmember_subl for specie in subl}
        real_species_dict = {abstract_specie: real_specie for abstract_specie, real_specie in
                             zip(endmember_speices, pmg.core.periodic_table._pt_data.keys())}
        # replace them
        endmember_subl = [[real_species_dict[specie] for specie in subl] for subl in endmember_subl]
        # get the structure and spacegroup info
        endmember_struct = self.get_concrete_sqs(endmember_subl, scale_volume=False)
        endmember_space_group_info = endmember_struct.get_space_group_info(symprec=symprec, angle_tolerance=angle_tolerance)
        return endmember_space_group_info

    def as_dict(self, verbosity=1, fmt=None, **kwargs):
        d = super(AbstractSQS, self).as_dict(verbosity=verbosity, fmt=fmt, **kwargs)
        d['sublattice_model'] = self.sublattice_model
        d['sublattice_names'] = self._sublattice_names
        d['sublattice_site_ratios'] = self.sublattice_site_ratios
        endmember_symmetry = self.get_endmember_space_group_info()
        d['symmetry'] = {'symbol': endmember_symmetry[0], 'number': endmember_symmetry[1]}
        return d

    @classmethod
    def from_dict(cls, d, fmt=None):
        sqs = super(AbstractSQS, cls).from_dict(d, fmt=fmt)
        sqs.sublattice_model = d.get('sublattice_model')
        sqs._sublattice_names = d.get('sublattice_names')
        return sqs


def enumerate_sqs(structure, subl_model, scale_volume=True, skip_on_failure=False):
    """
    Return a list of all of the concrete Structure objects from an abstract Structure and concrete sublattice model.
    Parameters
    ----------
    structure : AbstractSQS
        SQS object. Must be abstract.
    subl_model : [[str]]
        List of strings of species names, in the style of ESPEI `input.json`. This sublattice model
        can be of higher dimension than the SQS, e.g. a [["Al", "Fe", "Ni"]] for a fcc 75/25 binary SQS
        will generate the following Structures:
        Al0.75Fe0.25, Al0.75Ni0.25      Fe0.75Al0.25, Fe0.75Ni0.25      Ni0.75Al0.25, Ni0.75Fe0.25
        *Note that the ordering of species the sublattice model does not matter!*
    scale_volume : bool
        If True, scales the volume of the cell so the ions have at least their minimum atomic radii between them.
    skip_on_failure : bool
        If True, will skip if the sublattice model is lower order and return [] instead of raising

    Returns
    -------
    [PRLStructure]
        List of all concrete PRLStructure objects that can be created from the sublattice model.
    """
    if len(subl_model) != len(structure.sublattice_model):
        raise ValueError('Passed sublattice model ({}) does not agree with the passed structure ({})'.format(subl_model, structure.sublattice_model))
    possible_subls = []
    for subl, abstract_subl in zip(subl_model, structure.sublattice_model):
        subls = itertools.product(subl, repeat=len(abstract_subl))
        possible_subls.append(subls)
    unique_subl_models = itertools.product(*possible_subls)

    # create a list of unique concrete structures with the generated sublattice models
    unique_sqs = []
    unique_configurations_occupancies = []
    for model in unique_subl_models:
        proposed_sqs = structure.get_concrete_sqs(model, scale_volume)
        proposed_config_occupancy = (proposed_sqs.sublattice_configuration, proposed_sqs.sublattice_occupancies)
        if proposed_config_occupancy not in unique_configurations_occupancies:
            unique_configurations_occupancies.append(proposed_config_occupancy)
            unique_sqs.append(proposed_sqs)
    return unique_sqs


## Generating the ESPEI JSON from Pysipfenn 

### 1. Endmembers generation 

#### Load the POSCAR template of target phase

In [95]:
from pymatgen.core import Structure

with open("Laves_C15.POSCAR", 'r', encoding='utf-8') as file:
    POSCAR_STR = file.read()
structure = Structure.from_str(POSCAR_STR, fmt='POSCAR')

In [96]:
structure

Structure Summary
Lattice
    abc : 7.584994 7.584994 7.584994
 angles : 90.0 90.0 90.0
 volume : 436.3808910457692
      A : 7.584994 0.0 0.0
      B : 0.0 7.584994 0.0
      C : 0.0 0.0 7.584994
    pbc : True True True
PeriodicSite: Hf (3.7925, 0.0000, 3.7925) [0.5000, 0.0000, 0.5000]
PeriodicSite: Hf (1.8962, 1.8962, 1.8962) [0.2500, 0.2500, 0.2500]
PeriodicSite: Hf (3.7925, 3.7925, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Hf (1.8962, 5.6887, 5.6887) [0.2500, 0.7500, 0.7500]
PeriodicSite: Hf (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Hf (5.6887, 1.8962, 5.6887) [0.7500, 0.2500, 0.7500]
PeriodicSite: Hf (0.0000, 3.7925, 3.7925) [0.0000, 0.5000, 0.5000]
PeriodicSite: Hf (5.6887, 5.6887, 1.8962) [0.7500, 0.7500, 0.2500]
PeriodicSite: Hf (0.9481, 4.7406, 0.9481) [0.1250, 0.6250, 0.1250]
PeriodicSite: Hf (0.9481, 6.6369, 2.8444) [0.1250, 0.8750, 0.3750]
PeriodicSite: Hf (2.8444, 4.7406, 2.8444) [0.3750, 0.6250, 0.3750]
PeriodicSite: Hf (2.8444, 6.6369, 0.9481)

In [97]:
#Analyze sublattice inforamtion, please be careful about equivalent atoms with Wyckoff sites analyzed by pymatgen
prls=PRLStructure.from_structure(structure)
print('sublattice_site_ratios', prls.sublattice_site_ratios)
print('wyckoff_sites', prls.wyckoff_sites)
#wyckoff_site_list, sublattice_name=get_sublattice_information(structure, use_equivalent_atom=False)
wyckoff_site_list, sublattice_name, sublattice_site_ratio=get_sublattice_information(structure, use_equivalent_atom=True)
template_structure, template_configuration=get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
sublattice_configuration=[['Hf','Mo','Ti'],['Hf','Mo','Ti']]
comb, endmembers=get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)

sublattice_site_ratios [8, 16]
wyckoff_sites ['a', 'd']


### Generate and save the target phase endmembers into the directory 

In [98]:
import os

# Create the directory if it doesn't exist
directory_name = "LAVES_C15_endmembers"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

for i in range(0, len(endmembers)):
    filename = os.path.join(directory_name, str(i+1) +'_'+directory_name +'.POSCAR')
    endmembers[i].sort()
    endmembers[i].to(fmt='poscar', filename=filename)
    firstline = 'POSCAR' + ' ' + 'sublattice information' + ' ' + str(sublattice_name) + str(sublattice_site_ratio) + str(comb[i]) + ' ' + 'generate using OcMAT structure builders\n'
    #print(str(comb[i]))
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.readlines()
        data[0] = firstline
        
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(data)


### 2. HM_FROM result to ESPEI JSON file 

In [84]:
import json
import pandas as pd
import numpy as np
import itertools

In [99]:
Ternary_system = ['HF','MO','TI']
phase_name = "LAVES_C15"
sublattice_site_ratios = [1,2]
components= ['HF','MO','TI']

In [100]:
# Replace 'your_file.csv' with the actual path to your CSV file
csv_file = 'LAVES_C15_pySIPFENN.csv'
df = pd.read_csv(csv_file) # dft
df.head()

,Name,SIPFENN_Krajewski2022_NN30
0,1_LAVES_C15_endmembers.POSCAR,-0.021656
1,2_LAVES_C15_endmembers.POSCAR,-0.107798
2,3_LAVES_C15_endmembers.POSCAR,0.017813
3,4_LAVES_C15_endmembers.POSCAR,-0.064961
4,5_LAVES_C15_endmembers.POSCAR,0.222225


In [101]:
# Constant for conversion from eV/atom to J/mol-atom
conversion_factor = 96491

# Adding the new column
df["HM_FORM(J/mol-atom)"] = df["SIPFENN_Krajewski2022_NN30"] * conversion_factor
df

,Name,SIPFENN_Krajewski2022_NN30,HM_FORM(J/mol-atom)
0,1_LAVES_C15_endmembers.POSCAR,-0.021656,-2089.628063
1,2_LAVES_C15_endmembers.POSCAR,-0.107798,-10401.573020
2,3_LAVES_C15_endmembers.POSCAR,0.017813,1718.752566
3,4_LAVES_C15_endmembers.POSCAR,-0.064961,-6268.105245
4,5_LAVES_C15_endmembers.POSCAR,0.222225,21442.668905
5,6_LAVES_C15_endmembers.POSCAR,0.028094,2710.802086
6,7_LAVES_C15_endmembers.POSCAR,-0.044808,-4323.615848
7,8_LAVES_C15_endmembers.POSCAR,-0.001404,-135.516272
8,9_LAVES_C15_endmembers.POSCAR,-0.000550,-53.099975


In [102]:
# Creating a list and appending values from the HM_FORM(J/mol-atom) column
HM_FORM = df["HM_FORM(J/mol-atom)"].tolist()
HM_FORM

[-2089.628062698984,
 -10401.57302039116,
 1718.752565886825,
 -6268.105244807898,
 21442.668905362483,
 2710.802085962146,
 -4323.615848336368,
 -135.51627229991698,
 -53.099974506533215]

In [103]:
# Creating the list using list comprehension
configuration = [(comb[i]) for i in range(0, len(endmembers))]
configuration = [[element.upper() for element in sublist] for sublist in configuration]
configuration

[['HF', 'HF'],
 ['HF', 'MO'],
 ['HF', 'TI'],
 ['MO', 'HF'],
 ['MO', 'MO'],
 ['MO', 'TI'],
 ['TI', 'HF'],
 ['TI', 'MO'],
 ['TI', 'TI']]

In [104]:
dictionary1 ={
  "components": components,
  "phases": [phase_name],
  "solver": {
	  "mode": "manual",
	  "sublattice_site_ratios": sublattice_site_ratios,
	  "sublattice_configurations": configuration
  },
  "conditions": {
	  "P": 101325,
	  "T": 298.15
  },
  "output": "HM_FORM",
    "values":   [[HM_FORM]],
  "reference": "Endmembers_pySIPFEEN"
}
with open(phase_name+"_HMFORM_structure_pySIPFENN.json", "w") as outfile:
        json.dump(dictionary1, outfile,indent=1)

### Practice: try to run this process for the endmembers of LAVES_C36 phases

# SQS generation

#### For ternary BCC SQS structures, you can get the templates from PRL website: https://phaseslab.com/#/resources/
#### For Ternary HCP SQS structures, you can generature structures using ATAT SQS database

### Get HCP_A3 SQS from SQS ATAT database

In [157]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='HCP_A3'
system = 'Hf,Mo,Ti'
subl_model=[['Hf','Mo','Ti']]
subl_site_ratios=[1]

In [158]:
absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 1.0 0.9999996503124389 1.63299
 angles : 90.0 90.0 120.00001156757612
 volume : 1.41421016475
      A : 1.0 0.0 0.0
      B : -0.5 0.866025 0.0
      C : 0.0 0.0 1.63299
    pbc : True True True
PeriodicSite: Xca0+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Xca0+ (0.5000, 0.2887, 0.8165) [0.6667, 0.3333, 0.5000], 1: Structure Summary
Lattice
    abc : 4.242640687119285 2.0355803622677247 3.26598
 angles : 79.64109822073333 76.36697777463358 55.78303094338462
 volume : 22.627362636
      A : -1.0 1.0 4.0
      B : 1.366025 0.36602500000000004 1.4641000000000002
      C : 0.0 3.26598 0.0
    pbc : True True True
PeriodicSite: Xcc0+ (0.3660, 4.6320, 5.4641) [1.0000, 1.0000, 1.0000]
PeriodicSite: Xca0+ (0.3660, 2.9990, 5.4641) [1.0000, 1.0000, 0.5000]
PeriodicSite: Xca0+ (0.0458, 1.3955, 0.6830) [0.1250, 0.1250, 0.3750]
PeriodicSite: Xcc0+ (0.0458, 3.0285, 0.6830) [0.1250, 0.1250, 0.8750]
PeriodicSite: Xcc0+ (0.0915, 1.1580, 1.36

In [162]:
a=1
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][4]) #tenary hcp sqs: 1, 4 ; 1,7; 1,11; 5 4;

Full Formula (Hf16 Ti8 Mo8)
Reduced Formula: Hf2TiMo
abc   :  12.944474   6.210641   9.964641
angles:  79.641098  76.366978  55.783031
pbc   :       True       True       True
Sites (32)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Ti    1         1         1
  1  Hf    1         1         0.5
  2  Hf    0.125     0.125     0.375
  3  Ti    0.125     0.125     0.875
  4  Ti    0.25      0.25      0.25
  5  Hf    0.25      0.25      0.75
  6  Hf    0.375     0.375     0.125
  7  Hf    0.375     0.375     0.625
  8  Mo    0.5       0.5       1
  9  Hf    0.5       0.5       0.5
 10  Hf    0.625     0.625     0.375
 11  Mo    0.625     0.625     0.875
 12  Ti    0.75      0.75      0.25
 13  Hf    0.75      0.75      0.75
 14  Hf    0.875     0.875     0.125
 15  Hf    0.875     0.875     0.625
 16  Mo    0.729167  0.395834  0.104167
 17  Mo    0.729167  0.395834  0.604167
 18  Hf    0.854167  0.520833  0.479167
 19  Mo    0.854167  0.520833  0.9791

In [163]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [(1, 4), (1, 7), (1, 11), (5, 4)]

# Directory to save POSCAR files
directory_name = "SQS_HCP"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"HCPSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")


Saved POSCAR file for case (1, 4) as SQS_HCP/HCPSQS_1_4.POSCAR
Saved POSCAR file for case (1, 7) as SQS_HCP/HCPSQS_1_7.POSCAR
Saved POSCAR file for case (1, 11) as SQS_HCP/HCPSQS_1_11.POSCAR
Saved POSCAR file for case (5, 4) as SQS_HCP/HCPSQS_5_4.POSCAR


#### Try to generate the SQS BCC from ATAT database

In [132]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='BCC_A2'
system = 'Hf,Mo,Ti'
subl_model=[['Hf','Mo','Ti']]
subl_site_ratios=[1]


In [133]:
absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 3.1622776601683795 3.1622776601683795 3.3166247903554
 angles : 112.41950137364931 112.41950137364931 53.130102354155994
 volume : 24.0
      A : 0.0 -1.0 3.0
      B : 0.0 -3.0 1.0
      C : 3.0 1.0 -1.0
    pbc : True True True
PeriodicSite: Xaa0+ (3.0000, -1.0000, 2.0000) [0.8750, 0.3750, 1.0000]
PeriodicSite: Xaa0+ (3.0000, -2.0000, 2.0000) [0.7500, 0.7500, 1.0000]
PeriodicSite: Xab0+ (0.5000, -3.5000, 3.5000) [0.9167, 0.9167, 0.1667]
PeriodicSite: Xab0+ (3.0000, 0.0000, 1.0000) [0.6250, 0.1250, 1.0000]
PeriodicSite: Xaa0+ (3.0000, -1.0000, 1.0000) [0.5000, 0.5000, 1.0000]
PeriodicSite: Xac0+ (0.5000, -1.5000, 2.5000) [0.7917, 0.2917, 0.1667]
PeriodicSite: Xac0+ (3.0000, -2.0000, 1.0000) [0.3750, 0.8750, 1.0000]
PeriodicSite: Xaa0+ (0.5000, -2.5000, 2.5000) [0.6667, 0.6667, 0.1667]
PeriodicSite: Xab0+ (1.0000, -2.0000, 3.0000) [0.9583, 0.4583, 0.3333]
PeriodicSite: Xaa0+ (1.0000, -3.0000, 3.0000) [0.8333, 0.8333, 0.3333]
PeriodicSite: Xac0+ (

In [154]:
a=6
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][11]) #0 4;6 4 ;6 7; 6 11

Full Formula (Hf8 Ti16 Mo8)
Reduced Formula: HfTi2Mo
abc   :   9.484457   9.484457   8.213780
angles: 106.778655 106.778655  60.000000
pbc   :       True       True       True
Sites (32)
  #  SP         a       b      c
---  ----  ------  ------  -----
  0  Ti    0.125   0.125   0.75
  1  Mo    0.3125  0.3125  0.875
  2  Ti    0.125   0.625   0.75
  3  Mo    0.3125  0.8125  0.875
  4  Hf    0.0625  0.0625  0.375
  5  Ti    0.25    0.25    0.5
  6  Ti    0.0625  0.5625  0.375
  7  Mo    0.625   0.125   0.75
  8  Mo    0.4375  0.4375  0.625
  9  Ti    0.25    0.75    0.5
 10  Hf    0.8125  0.3125  0.875
 11  Ti    0.625   0.625   0.75
 12  Hf    0.4375  0.9375  0.625
 13  Hf    0.8125  0.8125  0.875
 14  Hf    1       1       1
 15  Mo    0.1875  0.1875  0.125
 16  Mo    1       0.5     1
 17  Ti    0.5625  0.0625  0.375
 18  Ti    0.375   0.375   0.25
 19  Ti    0.1875  0.6875  0.125
 20  Ti    0.75    0.25    0.5
 21  Ti    0.5625  0.5625  0.375
 22  Ti    0.375   0.875   0.25
 23  Mo 

In [155]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [[0,4],[6,4],[6,7],[6,11]]

# Directory to save POSCAR files
directory_name = "SQS_BCC"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"HCPSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")

Saved POSCAR file for case [0, 4] as SQS_BCC/HCPSQS_0_4.POSCAR
Saved POSCAR file for case [6, 4] as SQS_BCC/HCPSQS_6_4.POSCAR
Saved POSCAR file for case [6, 7] as SQS_BCC/HCPSQS_6_7.POSCAR
Saved POSCAR file for case [6, 11] as SQS_BCC/HCPSQS_6_11.POSCAR


In [164]:
Ternary_system = ['HF','MO','TI']
phase_name = "HCP_A3"
sublattice_site_ratios = [1,0.5]
components= ['HF','MO','TI', 'VA']

In [165]:
# Replace 'your_file.csv' with the actual path to your CSV file
csv_file = 'SQS_HCP_pySIPFENN.csv'
df = pd.read_csv(csv_file) # dft
#df.head()
# Constant for conversion from eV/atom to J/mol-atom
conversion_factor = 96491

# Adding the new column
df["HM_FORM(J/mol-atom)"] = df["SIPFENN_Krajewski2022_NN30"] * conversion_factor
# Creating a list and appending values from the HM_FORM(J/mol-atom) column
HM_FORM = df["HM_FORM(J/mol-atom)"].tolist()
HM_FORM

[23974.695418700576, 43116.143703728914, 26273.093582779165, 27269.1977776587]

In [166]:
configuration_1 = [['HF','MO','TI'],'VA']
configuration_2 = [['HF','MO','TI'],'VA']
configuration_3 = [['HF','MO','TI'],'VA']
configuration_4 = [['HF','MO','TI'],'VA']
configuration = [configuration_1,configuration_2,configuration_3,configuration_4]

sublattice_occupancies_1 = [[0.5,0.25,0.25],1]
sublattice_occupancies_2 = [[0.25,0.5,0.25],1]
sublattice_occupancies_3 = [[0.25,0.25,0.5],1]
sublattice_occupancies_4 = [[0.333,0.333,0.334],1]
sublattice_occupancies = [sublattice_occupancies_1,sublattice_occupancies_2,sublattice_occupancies_3,sublattice_occupancies_4]

In [168]:
# Define the elements and their state
elements = ['HF', 'MO', 'TI']
state = 'VA'

# Create unique configurations by combining elements with the state
configuration_1 = [elements, state]
configuration_2 = [elements, state]
configuration_3 = [elements, state]
configuration_4 = [elements, state]
configurations = [configuration_1, configuration_2, configuration_3, configuration_4]

# Define the sublattice occupancies
sublattice_occupancies_1 = [[0.5, 0.25, 0.25], 1]
sublattice_occupancies_2 = [[0.25, 0.5, 0.25], 1]
sublattice_occupancies_3 = [[0.25, 0.25, 0.5], 1]
sublattice_occupancies_4 = [[0.333, 0.333, 0.334], 1]
sublattice_occupancies = [sublattice_occupancies_1, sublattice_occupancies_2, sublattice_occupancies_3, sublattice_occupancies_4]

# Display the configurations and sublattice occupancies
print("Configurations:", configurations)
print("Sublattice Occupancies:", sublattice_occupancies)



Configurations: [[['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA']]
Sublattice Occupancies: [[[0.5, 0.25, 0.25], 1], [[0.25, 0.5, 0.25], 1], [[0.25, 0.25, 0.5], 1], [[0.333, 0.333, 0.334], 1]]


In [169]:
dictionary1 ={
  "components": components,
  "phases": [phase_name],
  "solver": {
	  "mode": "manual",
	  "sublattice_site_ratios": sublattice_site_ratios,
      "sublattice_occupancies": sublattice_occupancies,
	  "sublattice_configurations": configuration
  },
  "conditions": {
	  "P": 101325,
	  "T": 298.15
  },
  "output": "HM_FORM",
    "values":   [[HM_FORM]],
  "reference": "SQS_SIPFEEN"
}
with open(phase_name+"_HMFORM_sqs_structure_SIPFENN.json", "w") as outfile:
        json.dump(dictionary1, outfile,indent=1)